In [12]:
import json
import os
from typing import Dict, Any


class PDFParser:
    def parse(
        self, template: Dict[str, Any], pdf_data: Dict[str, Any]
    ) -> Dict[str, Any]:
        pass


template_path: str = os.path.join("src", "templates", "barclays_template.json")
pdf_data_path: str = os.path.join("src", "pdf_data", "barclays_pdf_data.json")

template: Dict[str, Any] = json.load(open(template_path))
pdf_data: Dict[str, Any] = json.load(open(pdf_data_path))

parser: PDFParser = PDFParser()
output_generated: Dict[str, Any] = parser.parse(template, pdf_data)

output_data = json.load(open("src/outputs/barclays_output.json"))


In [13]:
# Create a FormParser

[{'page_numbers': ['1'],
  'forms': ['customer_details', 'sort_code', 'account_number']},
 {'page_numbers': ['2'],
  'forms': ['start_balance',
   'money_in',
   'money_out',
   'end_balance',
   'overdraft_limit']},
 {'page_numbers': ['2-(-2)'],
  'forms': [],
  'tables': ['transactions_page_2_onwards']},
 {'page_numbers': ['-1'], 'ignore': True}]

In [27]:
def get_items_in_bounding_box(page_data, coordinates):
    items_in_box = []
    for item in page_data:
        bounding_box = item["bounding_box"]["decimal_coordinates"]
        if (
            bounding_box["top_left"]["x"] >= coordinates["top_left"]["x"]
            and bounding_box["top_left"]["y"] >= coordinates["top_left"]["y"]
            and bounding_box['bottom_right']['x'] <= coordinates['bottom_right']['x']
            and bounding_box['bottom_right']['y'] <= coordinates['bottom_right']['y']
        ):
            items_in_box.append(item)
    return items_in_box



def get_text_from_items(items):
    return "".join([item["text"] for item in items])


def get_text_from_page(page_content, coordinates):
    items_within_coordinates = get_items_in_bounding_box(page_content, coordinates)
    return get_text_from_items(items_within_coordinates)


def get_output_data_for_rule(form_rule, page_number):
    coordinates = form_rule["config"]["coordinates"]
    page_content = pdf_data["pages"][page_number-1]["content"]
    return {form_rule["config"]["field_name"]: get_text_from_page(page_content, coordinates)}

In [41]:
page_number = 1

form_rule = template["rules"][0]



output_data_for_rule = get_output_data_for_rule(form_rule, page_number)


{'sort_code': '20-44-51'}
